In [ ]:
!pip install "git+https://github.com/nixtla/neuralforecast.git@main"
!pip install darts

In [ ]:
# import packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from neuralforecast import NeuralForecast
from neuralforecast.losses.pytorch import MQLoss

from datetime import date
from darts import TimeSeries, concatenate
from darts.dataprocessing.transformers import Scaler
from darts.utils.statistics import check_seasonality, extract_trend_and_seasonality
from darts.metrics import mape, rmse, mae, smape
from darts.utils.timeseries_generation import datetime_attribute_timeseries, holidays_timeseries
from darts.utils.likelihood_models import QuantileRegression

In [ ]:
Y_df = pd.read_csv('Y_df.csv')

Y_df['datetime'] = pd.to_datetime(Y_df['ds'])
Y_df.drop('ds', axis=1, inplace=True)
Y_df.set_index('datetime', inplace=True)
series = TimeSeries.from_series(Y_df)

series = series.add_datetime_attribute('hour')
series = series.add_datetime_attribute('dayofweek')
series = series.add_datetime_attribute('month')
series = series.add_datetime_attribute('quarter')
series = series.add_datetime_attribute('day')
series = series.add_datetime_attribute('year')
series = series.add_holidays(country_code='ITA')

Y_df = TimeSeries.pd_dataframe(series).reset_index()
Y_df.rename({'datetime': 'ds'}, axis=1, inplace=True)
Y_df

In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
Y_df.reset_index(inplace=True)
def is_bridge_day(day, holiday, dayofweek):
    if holiday == 1:
        return 7
    elif dayofweek in [5,6] and holiday == 0:
        return dayofweek
    elif dayofweek == 0 and holiday == 0 and Y_df.iloc[day+24]['holidays'] == 1:
        return 8
    elif dayofweek == 4 and holiday == 0 and Y_df.iloc[day-24]['holidays'] == 1:
        return 8
    else:
        return dayofweek

Y_df['day_of_week'] = np.vectorize(is_bridge_day)(Y_df.index, Y_df['holidays'], Y_df['dayofweek'])

encoder = LabelEncoder()

calendar = ['hour', 'day', 'day_of_week', 'month', 'year']
for cal in calendar:
  Y_df[cal] = encoder.fit_transform(Y_df[cal]).astype(np.int32)
Y_df

In [ ]:
val_size  = 190*24
test_size = 190*24
Y_train_df = Y_df.iloc[:-test_size, :]
Y_test_df = Y_df.iloc[-test_size:, :]
Y_val_df = Y_train_df.iloc[-val_size:, :]

In [ ]:
# Create the plot
import matplotlib
matplotlib.rc_file_defaults()

# Create subplots with two rows
fig, axs = plt.subplots(3, 1, figsize=(15, 12), sharex=True)

# Plot the load data in the first subplot
axs[0].plot(Y_train_df['ds'], Y_train_df['y'], color='steelblue', label='Electricity Price')
axs[0].plot(Y_val_df['ds'], Y_val_df['y'], color='steelblue')
axs[0].plot(Y_test_df['ds'], Y_test_df['y'], color='steelblue')
axs[0].set_ylabel('Spot Price (€/MWh)', fontsize=12)
axs[0].legend(loc = "upper left")
legend = axs[0].get_legend()

# Set the font size of the legend
legend.get_frame().set_facecolor('white')  # Optional: Set legend background color
legend.get_frame().set_linewidth(0.5)  # Optional: Set legend frame linewidth
for text in legend.get_texts():
    text.set_fontsize(12)  # Set the font size


# Plot the temperature data in the second subplot
axs[1].plot(Y_train_df['ds'], Y_train_df['psvda'], color='seagreen', label='PSVDA')
axs[1].plot(Y_val_df['ds'], Y_val_df['psvda'], color='seagreen')
axs[1].plot(Y_test_df['ds'], Y_test_df['psvda'], color='seagreen')
axs[1].set_ylabel('PSV day-ahead (€/MWh)', fontsize=12)
axs[1].legend(loc="upper left")
# Get the current legend
legend_1 = axs[1].get_legend()

# Set the font size of the legend
legend_1.get_frame().set_facecolor('white')  # Optional: Set legend background color
legend_1.get_frame().set_linewidth(0.5)  # Optional: Set legend frame linewidth
for text in legend_1.get_texts():
    text.set_fontsize(12)  # Set the font size

# Plot the temperature data in the second subplot
axs[2].plot(Y_train_df['ds'], Y_train_df['load forecast'], color='lightcoral', label='Load Forecast')
axs[2].plot(Y_val_df['ds'], Y_val_df['load forecast'], color='lightcoral')
axs[2].plot(Y_test_df['ds'], Y_test_df['load forecast'], color='lightcoral')
axs[2].set_ylabel('Load (MW)', fontsize=12)
axs[2].legend(loc="upper left")
# Get the current legend
legend_2 = axs[2].get_legend()

# Set the font size of the legend
legend_2.get_frame().set_facecolor('white')  # Optional: Set legend background color
legend_2.get_frame().set_linewidth(0.5)  # Optional: Set legend frame linewidth
for text in legend_2.get_texts():
    text.set_fontsize(12)  # Set the font size


# Add annotations for the splits
axs[0].annotate('Training', xy=(Y_train_df['ds'].mean(), Y_train_df['y'].max()), xytext=(0, 20),
             xycoords='data', textcoords='offset points', fontsize=15, ha='center')
axs[0].annotate('Validation', xy=(Y_val_df['ds'].mean(), Y_train_df['y'].max()), xytext=(0, 20),
             xycoords='data', textcoords='offset points', fontsize=15, ha='center')
axs[0].annotate('Test', xy=(Y_test_df['ds'].mean(), Y_train_df['y'].max()), xytext=(0, 20),
             xycoords='data', textcoords='offset points', fontsize=15, ha='center')

# Add dashed lines for the splits
axs[0].axvline(Y_val_df['ds'].iloc[0], color='k', linestyle='--')
axs[0].axvline(Y_val_df['ds'].iloc[-1], color='k', linestyle='--')
axs[1].axvline(Y_val_df['ds'].iloc[0], color='k', linestyle='--')  # Shared vertical line
axs[1].axvline(Y_val_df['ds'].iloc[-1], color='k', linestyle='--')
axs[2].axvline(Y_val_df['ds'].iloc[0], color='k', linestyle='--')  # Shared vertical line
axs[2].axvline(Y_val_df['ds'].iloc[-1], color='k', linestyle='--')

plt.xlabel('Datetime', fontsize=12)
#plt.suptitle('Electricity Demand', fontsize=15)
plt.subplots_adjust(hspace=0)
plt.show()

In [ ]:

# Create a copy of Y_train_df
df_train = Y_train_df.copy()

# Map quarter labels to the "Quarter" column
quarter_labels = {1: 'Q1', 2: 'Q2', 3: 'Q3', 4: 'Q4'}
df_train['Quarter'] = df_train['quarter'].map(quarter_labels)

# Create the plot
plt.figure(figsize=(10, 7))
sns.lineplot(data=df_train, x='hour', y='y', hue='Quarter', palette=palette, linewidth=3)

# Remove the spines
sns.despine()

# Set the title
plt.ylabel('Price (€/MWh)')

# Change the legend labels
legend = plt.legend(fontsize='large')
for line, label in zip(legend.get_lines(), quarter_labels.values()):
    line.set_linewidth(3.0)
    line.set_label(label)
plt.grid(False)
# Show the plot
plt.show()


In [ ]:
from neuralforecast.losses.pytorch import MQLoss
from neuralforecast.auto import AutoNBEATSx
from ray import tune
horizon=24
levels=[80,90]
# Use your own config or AutoLSTM.default_config
nbeats_config = {
       "futr_exog_list" : tune.choice([['hour', 'dayofweek', 'month', 'quarter', 'holidays', 'day', 'year'],
                                       ['hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'dayofweek_sin', 'dayofweek_cos',
                                        'month_sin', 'month_cos', 'quarter_sin', 'quarter_cos', 'year', 'holidays']]),
       "learning_rate": tune.choice([1e-3, 1e-4]),
       "max_steps": tune.choice([100, 200, 400]),
       "input_size": tune.choice([5*horizon, 7*horizon, 2*horizon]),
       "batch_size": tune.choice([32, 64]),
       "activation": tune.choice(['ReLU']),
       "n_blocks":  tune.choice([[1, 2, 3], [1, 1, 1]]),
       "mlp_units":  tune.choice([[[512, 512], [512, 512], [512, 512]]]),
       "val_check_steps": tune.choice([20]),
       "random_seed": tune.randint(1, 10),
       "scaler_type": tune.choice(['minmax', 'robust'])
    }


model_nbeats = [AutoNBEATSx(h=horizon,
                   config=nbeats_config,
                   loss=MQLoss(level=levels),
                   num_samples=10,
                   verbose=True,
                   refit_with_val=True)]

In [ ]:
nf = NeuralForecast(models=model_nbeats, freq='H')
fcst_df = nf.fit(Y_train_df, val_size=val_size, verbose=True)

In [ ]:
from neuralforecast.models import NBEATSx
from neuralforecast.losses.pytorch import MQLoss
from neuralforecast.auto import AutoNBEATSx
from ray import tune
levels = [90]
model_nbeats = NBEATSx(h=24,
            input_size=168,
            learning_rate=0.001,
            max_steps=400,
            val_check_steps=20,
            loss=MQLoss(level=levels),
            batch_size=64,
            activation = 'ReLU',
            n_blocks = [1, 1, 1],
            mlp_units = [[512, 512], [512, 512], [512, 512]],
            random_seed = 3,
            scaler_type = 'minmax',
            futr_exog_list =['hour', 'day_of_week', 'month',  'day', 'year', 'load forecast'],
            hist_exog_list = ['psvda'])

fcst = NeuralForecast(
    models=[model_nbeats],
    freq='H'
)
fcst.fit(Y_train_df, verbose=True)

In [ ]:
#Y_df['dayofweek'] = Y_df['dayofweek'].astype('float32')
Y_test_df['date'] = [x[:11] for x in Y_test_df.ds.astype('str')]
Y_df['date'] = [x[:11] for x in Y_df.ds.astype('str')]

In [ ]:
Y_hat_df_nbeats = pd.DataFrame()
for day in Y_test_df['date'].drop_duplicates():
    df_futr = Y_df[Y_df['date'] == day]
    df_test = Y_df[Y_df['date'] < day].tail(len(Y_train_df)) # the size of training df remain the same
    check_fit = df_futr.iloc[:1, :]['dayofweek'].values[0]
    #if check_fit == 3.0:
       #fcst.fit(df_test.drop('date', axis=1), verbose=True)
    Y_hat = fcst.predict(df=df_test.drop('date', axis=1), futr_df=df_futr.drop('date', axis=1))
    Y_hat_df_nbeats = pd.concat([Y_hat_df_nbeats, Y_hat], axis=0)
Y_hat_df_nbeats

In [ ]:
from neuralforecast.losses.numpy import mae, mape
Y_hat_df_nbeats['ds'] = pd.to_datetime(Y_hat_df_nbeats['ds'])
Y_hat_df_nbeats.columns = Y_hat_df_nbeats.columns.str.replace('-median', '')
plot_df_nbeats = Y_test_df.merge(Y_hat_df_nbeats, on=['unique_id','ds'], how='inner')
plt.figure(figsize=(20,5))
plt.plot(plot_df_nbeats['ds'], plot_df_nbeats['y'], c='black', label='True')
plt.plot(plot_df_nbeats['ds'], plot_df_nbeats['NBEATSx'], c='steelblue', label='Forecast')
#plt.axvline(pd.to_datetime('2022-12-24'), color='red', linestyle='-.')
plt.fill_between(x=plot_df_nbeats['ds'],
                    y1=plot_df_nbeats['NBEATSx-lo-90'], y2=plot_df_nbeats['NBEATSx-hi-90'],
                    alpha=0.4, label='level 90', color='steelblue')
plt.legend()
plt.grid()
plt.plot()

print('MAE: ', mae(plot_df_nbeats['y'], plot_df_nbeats['NBEATSx']))
print('MAPE: ', mape(plot_df_nbeats['y'], plot_df_nbeats['NBEATSx']))